In [1]:
!sudo pip3 install tensorflow

    100% |████████████████████████████████| 109.2MB 14kB/s  eta 0:00:01 8% |██▉                             | 9.8MB 22.1MB/s eta 0:00:05    40% |█████████████                   | 44.2MB 17.8MB/s eta 0:00:04
    100% |████████████████████████████████| 51kB 6.7MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 7.1MB/s eta 0:00:01
    100% |████████████████████████████████| 20.8MB 78kB/s  eta 0:00:01    61% |███████████████████▋            | 12.8MB 46.1MB/s eta 0:00:01
    100% |████████████████████████████████| 13.4MB 121kB/s eta 0:00:01   14% |████▊                           | 2.0MB 25.0MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 7.5MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 10.3MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 3.2MB 526kB/s eta 0:00:01
    100% |████████████████████████████████| 491kB 3.3MB/s eta 0:00:01
    100% |████████████████

In [2]:
!sudo pip3 install h5py

In [3]:
!sudo pip3 install numpy

In [4]:
!sudo pip3 install pandas

    100% |████████████████████████████████| 10.5MB 157kB/s eta 0:00:01


In [5]:
!sudo pip3 install --upgrade pip

    100% |████████████████████████████████| 1.5MB 1.1MB/s eta 0:00:01


In [6]:
!sudo pip3 install opencv-python

     |████████████████████████████████| 49.5 MB 85 kB/s s eta 0:00:01


In [7]:
!sudo pip3 install matplotlib

     |████████████████████████████████| 11.6 MB 2.2 MB/s eta 0:00:01    |█████████████▎                  | 4.8 MB 2.2 MB/s eta 0:00:04
     |████████████████████████████████| 88 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 28.5 MB/s eta 0:00:01


In [8]:
!sudo pip3 install keras==2.3.1

     |████████████████████████████████| 377 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 21.7 MB/s eta 0:00:01     |███████████████▎                | 12.3 MB 16.7 MB/s eta 0:00:01


In [9]:
!sudo pip3 install scikit-learn

     |████████████████████████████████| 6.8 MB 2.9 MB/s eta 0:00:01     |████                            | 870 kB 2.9 MB/s eta 0:00:03     |█████▋                          | 1.2 MB 2.9 MB/s eta 0:00:02     |██████████▏                     | 2.2 MB 2.9 MB/s eta 0:00:02     |████████████▊                   | 2.7 MB 2.9 MB/s eta 0:00:02
     |████████████████████████████████| 300 kB 10.6 MB/s eta 0:00:01


In [10]:
!sudo pip3 install ray

     |████████████████████████████████| 22.9 MB 51.0 MB/s eta 0:00:01   |███████▊                        | 5.5 MB 2.0 MB/s eta 0:00:09     |████████▍                       | 6.0 MB 2.0 MB/s eta 0:00:09     |█████████████████▎              | 12.3 MB 2.0 MB/s eta 0:00:06
     |████████████████████████████████| 82 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 274 kB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 201 kB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 257 kB 43

  Created wheel for psutil: filename=psutil-5.7.2-cp36-cp36m-linux_x86_64.whl size=266563 sha256=c43d22689c998a51bf7909300f75f5b8c38c94618e22219770770336e9b83bf2
  Stored in directory: /root/.cache/pip/wheels/4c/1f/18/c066f071b2c2b638e40889c5595bc6b61fbe45eb475d29d61d
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7665 sha256=0ae860f7a06f6e5175673519f164c868458d1f2283a45e421bf7f1846b2462c2
  Stored in directory: /root/.cache/pip/wheels/41/11/53/911724983aa48deb94792432e14e518447212dd6c5477d49d3
Successfully built gpustat idna-ssl nvidia-ml-py3 psutil contextvars
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-auth 1.22.0 requires setuptools>=40.3.0, but you'll have setuptools 39.2.0 which is incompatible

In [11]:
!sudo pip3 install ray[tune]

     |████████████████████████████████| 308 kB 4.2 MB/s eta 0:00:01


In [12]:
!sudo pip3 install tensorflow==2.2

     |████████████████████████████████| 516.2 MB 27 kB/s s eta 0:00:01   |█▏                              | 19.3 MB 3.9 MB/s eta 0:02:09     |█▌                              | 24.1 MB 6.9 MB/s eta 0:01:12     |█▋                              | 26.3 MB 6.9 MB/s eta 0:01:11     |███                             | 49.2 MB 13.0 MB/s eta 0:00:36     |███▉                            | 61.2 MB 10.5 MB/s eta 0:00:44     |████▏                           | 67.3 MB 10.5 MB/s eta 0:00:43     |█████▍                          | 87.4 MB 5.0 MB/s eta 0:01:252     |████████▉                       | 141.9 MB 48.3 MB/s eta 0:00:08     |██████████                      | 159.9 MB 52.7 MB/s eta 0:00:07     |███████████▋                    | 188.1 MB 41.8 MB/s eta 0:00:08     |██████████████████████████▌     | 427.7 MB 35.8 MB/s eta 0:00:03
     |████████████████████████████████| 3.0 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 3.4 MB/s eta 0:00:01
     |█████████████████████████████

  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.2.0
    Uninstalling setuptools-39.2.0:
      Successfully uninstalled setuptools-39.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [13]:
!sudo pip3 install keras

In [92]:
!sudo pip3 install ray[rllib]

     |████████████████████████████████| 1.8 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 15.7 MB/s eta 0:00:01
     |████████████████████████████████| 36.4 MB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 28.1 MB/s eta 0:00:01


  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654652 sha256=6956e858969f1c0ef590d66911c1c4bed92aa5e01663a344210775333c778970
  Stored in directory: /root/.cache/pip/wheels/95/b0/62/af38051b97354eab5b2ff9a5fb92f5015a480745568c91e0b0
Successfully built gym


In [14]:
#!/usr/bin/env python3
import numpy as np 
import tensorflow as tf
import pandas as pd
import os
from cv2 import imread, createCLAHE 
import cv2
import pickle
from glob import glob
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from keras import backend as keras
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam 
from ray import tune

class UNet:
    def DataLoader(self):
        infile = open("/home/scitech/shared-data/mask_wf/op/data_split.pkl",'rb')
        new_dict = pickle.load(infile)
        infile.close()

        path = "/home/scitech/shared-data/mask_wf/op/"

        train_data = new_dict['train']
        valid_data = new_dict['valid']
        test_data = new_dict['test']

        X_train = [cv2.imread(os.path.join(path,i))[:,:,0] for i in train_data if 'mask' not in i]
        y_train = [cv2.imread(os.path.join(path,i))[:,:,0] for i in train_data if 'mask' in i]

        X_valid = [cv2.imread(os.path.join(path,i))[:,:,0] for i in valid_data if 'mask' not in i]
        y_valid = [cv2.imread(os.path.join(path,i))[:,:,0] for i in valid_data if 'mask' in i]

        X_test = [cv2.imread(os.path.join(path,i))[:,:,0] for i in test_data]

        dim = 256

        X_train = np.array(X_train).reshape((len(X_train),dim,dim,1))
        y_train = np.array(y_train).reshape((len(y_train),dim,dim,1))
        X_valid = np.array(X_valid).reshape((len(X_valid),dim,dim,1))
        y_valid = np.array(y_valid).reshape((len(y_valid),dim,dim,1))
        X_test = np.array(X_test).reshape((len(X_test),dim,dim,1))
        assert X_train.shape == y_train.shape
        assert X_valid.shape == y_valid.shape

        return X_train.astype(np.float32), y_train.astype(np.float32), X_valid.astype(np.float32), y_valid.astype(np.float32)
    
    def model(self, input_size=(256,256,1)):
        inputs = Input(input_size)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

        up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
        conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
        conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

        up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
        conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
        conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

        up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
        conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
        conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

        up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
        conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
        conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

        conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

        return Model(inputs=[inputs], outputs=[conv10])

def dice_coef(y_true, y_pred):
        y_true_f = keras.flatten(y_true)
        y_pred_f = keras.flatten(y_pred)
        intersection = keras.sum(y_true_f * y_pred_f)
        return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)
    
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


class TuneReporterCallback(Callback):
    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        tune.report(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))

def tune_unet(config):
    unet = UNet()
    model = unet.model()
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='loss', save_best_only=True, save_freq=2)
    callbacks = [checkpoint_callback, TuneReporterCallback()]
    model.compile(optimizer=Adam(lr=config["lr"]), loss=[dice_coef_loss], metrics = [dice_coef, 'binary_accuracy'])
    train_vol, train_seg, valid_vol, valid_seg = unet.DataLoader()
    loss_history = model.fit(x = train_vol, y = train_seg, batch_size = 16, epochs = 5, validation_data =(valid_vol, valid_seg), callbacks = callbacks)
    
    return loss_history

import numpy as np
import ray
np.random.seed(5)  
hyperparameter_space = {
    "lr": tune.loguniform(0.0002, 0.2)
    }

TRIALS = 3

ray.shutdown()  
ray.init(log_to_driver=False)
! rm -rf ~/ray_results/tune_unet

analysis = tune.run(
    tune_unet, 
    verbose=1, 
    config=hyperparameter_space,
    num_samples=TRIALS)

df = analysis.dataframe()

Trial name,status,loc,lr,loss,iter,total time (s),neg_mean_loss
tune_unet_d1b90_00000,TERMINATED,,0.00092685,-1.96922,5,638.149,1.96922
tune_unet_d1b90_00001,TERMINATED,,0.0818893,-1.96905,5,639.399,1.96905
tune_unet_d1b90_00002,TERMINATED,,0.000834041,-1.96922,5,627.955,1.96922


ValueError: No `metric` has been passed and  `default_metric` has not been set. Please specify the `metric` parameter.

In [85]:
df.to_csv("tune_unet.csv", sep='\t', encoding='utf-8')

In [87]:
df

,mean_accuracy,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,date,...,trial_id,experiment_tag,keras_info/loss,keras_info/dice_coef,keras_info/binary_accuracy,keras_info/val_loss,keras_info/val_dice_coef,keras_info/val_binary_accuracy,config/lr,logdir
0,NaN,-1.969369,156.057521,False,NaN,NaN,5,1.969369,0a46962a9fdf4a47bd947e3d93a42ef0,2020-10-01_04-49-26,...,480e8_00000,0_lr=0.00092685,-1.969369,1.969413,0.004293,-1.969349,1.969511,0.013983,0.000927,/home/scitech/ray_results/tune_unet/tune_unet_...
1,NaN,-1.969195,156.030143,False,NaN,NaN,5,1.969195,a50c42ed28544f93b603eac6616b6ca2,2020-10-01_04-49-23,...,480e8_00001,1_lr=0.081889,-1.969195,1.968757,0.000032,-1.968927,1.968831,0.000023,0.081889,/home/scitech/ray_results/tune_unet/tune_unet_...
2,NaN,-1.969351,156.931784,False,NaN,NaN,5,1.969351,fcee674f0996454eaca4585f9b4433e4,2020-10-01_04-49-29,...,480e8_00002,2_lr=0.00083404,-1.969351,1.969484,0.000380,-1.969180,1.969170,0.002304,0.000834,/home/scitech/ray_results/tune_unet/tune_unet_...
